In [53]:
from typing import List, Dict, AsyncGenerator
# from pytablewriter import MarkdownTableWriter
# from pytablewriter.style import Style
import json
import dspy
import faiss
from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS as LangchainFAISS
import numpy as np
import pandas as pd


In [54]:
import ipywidgets as widgets
from IPython.display import display

In [93]:
from pytablewriter import MarkdownTableWriter
from pytablewriter.style import Style

In [55]:
faiss_index_path_qsr = '../../../../web/db/20250317/faiss_index_qsrc.faiss'
faiss_index_path_module = '../../../../web/db/20250317/faiss_index_module.faiss';
# datasrc_deqlearn = os.getenv("DATASRC_DEQ_LEARN");
datasrc_deqaitrial = '../../../../doc/DQE_Lesson_learn_AI_0311_five_refine_question_AI_Trial_Run.csv'
local_embedding_path = "/home/mapleleaf/LCJRepos/Embedding_Models/paraphrase-multilingual-MiniLM-L12-v2/"

In [56]:
class CustomFAISSRetriever(dspy.Retrieve):
    def __init__(self, faiss_index_path:str=None, vector_db_path:str=None, model_name:str=None, k=3):
        super().__init__()
        self.k = k
        # 使用同一個模型名稱
        self.model_name = model_name;#"paraphrase-multilingual-MiniLM-L12-v2"
        print(f"start to insitalize sentence embeddings : {local_embedding_path}");
        # 初始化 embeddings
        self.embeddings = HuggingFaceEmbeddings(
            model_name=local_embedding_path#self.model_name
        )
        print(f"embedding has been insitalized : {local_embedding_path}");
        # 載入 FAISS 索引
        self.index = self.load_index(faiss_index_path)
        
        # 載入向量庫
        if vector_db_path != "nodata":
            self.vector_db = self.load_local_db(vector_db_path, self.embeddings)
        
        # 使用相同的模型進行查詢編碼
        self.model = SentenceTransformer(local_embedding_path)

    def load_index(self, idx_path=None, model_name:str=None):
        try:
            index = faiss.read_index(idx_path)
            print(f"成功載入FAISS索引，包含 {index.ntotal} 個向量")
            return index
        except Exception as e:
            print(f"索引載入失敗: {str(e)}")
            return None

    def load_local_db(self, local_db_path=None, embeddings=None):
        # try:
        if local_db_path=="nodata":
            return "nodata";
        db = LangchainFAISS.load_local(
            folder_path=local_db_path,
            embeddings=embeddings,
            allow_dangerous_deserialization=True
        )
        print(f"載入成功，共 {db.index.ntotal} 筆技術問答")
        return db
        # except Exception as e:
        #     print(f"向量庫載入異常: {str(e)}")
        #     return None

    def __call__(self, query):
        # 編碼查詢
        query_embedding = self.model.encode(
            query,
            convert_to_tensor=False,
            show_progress_bar=False  # 對單一查詢關閉進度條
        )
        query_embedding = query_embedding.reshape(-1,1).T
        # query_embedding = query_embedding.cpu().numpy()
        query_embedding = query_embedding.astype(np.float32)
        # 搜索向量庫
        # docs = self.vector_db.similarity_search_with_score(query_embedding, k=self.k)
        distance,pos = self.index.search(query_embedding, k=self.k)
        print(distance)
        print(pos)
        # return the pos for retrieving data from answers
        return pos, distance;

In [57]:
def gen_data_src_list(dict_data=None):
    row_list = [v for v in dict_data.values]
    return row_list
    
def sanitize_text(text):
    if pd.isna(text):
        return ""
    text = str(text).strip()
    # 保留必要的符號
    special_chars = ['|', '-', ':', '<br>']
    for char in special_chars:
        text = text.replace(f' {char} ', char)
    # 移除多餘的空格
    text = ' '.join(text.split())
    return text


In [58]:
def generate_markdown_table_from_2DMatrix(headers, value_matrix):
    writer = MarkdownTableWriter(
        tablename="回覆表格",
        headers=headers,
        value_matrix=value_matrix,
        column_alignments=["left"] * len(headers),  # Explicit alignment
        margin=1  # Add margin for better readability
    )
    # Add table style
    writer.style = Style(
        align_header="center",
        border_chars={
            "left": "|",
            "right": "|",
            "header": "-",
            "center": "|",
        }
    )
    return writer.dumps()

In [59]:
def search_similar_data(retriever, question):
    pos, distances = retriever(question)
    return pos, distances

In [60]:
faiss_retriever_qsrc=CustomFAISSRetriever(faiss_index_path=faiss_index_path_qsr, vector_db_path="nodata", k=3);
faiss_retriever_module = CustomFAISSRetriever(faiss_index_path=faiss_index_path_module, vector_db_path="nodata", k=3);

start to insitalize sentence embeddings : /home/mapleleaf/LCJRepos/Embedding_Models/paraphrase-multilingual-MiniLM-L12-v2/
embedding has been insitalized : /home/mapleleaf/LCJRepos/Embedding_Models/paraphrase-multilingual-MiniLM-L12-v2/
成功載入FAISS索引，包含 5 個向量
start to insitalize sentence embeddings : /home/mapleleaf/LCJRepos/Embedding_Models/paraphrase-multilingual-MiniLM-L12-v2/
embedding has been insitalized : /home/mapleleaf/LCJRepos/Embedding_Models/paraphrase-multilingual-MiniLM-L12-v2/
成功載入FAISS索引，包含 5 個向量


In [61]:
question_txt = None;
# 創建文本輸入框
text_input = widgets.Text(
    description='輸入:',
    placeholder='請輸入資料'
)
# 創建按鈕
button = widgets.Button(description='確認')
# 顯示輸入結果的標籤
output = widgets.Label(value='等待輸入...')
# 定義按鈕點擊事件
def on_button_click(b):
    question_txt = text_input.value
    output.value = f'您輸入的是: {text_input.value}'

button.on_click(on_button_click)

# 顯示所有元件
display(text_input, button, output)

Text(value='', description='輸入:', placeholder='請輸入資料')

Button(description='確認', style=ButtonStyle())

Label(value='等待輸入...')

In [62]:
_pos_lists_qsrc, _dist_lists_qsrc = search_similar_data(retriever=faiss_retriever_qsrc, question='svd') 

[[4.8148732e+00 3.4028235e+38 3.4028235e+38]]
[[ 3 -1 -1]]


In [63]:
_pos_lists_module, _dist_lists_module = search_similar_data(retriever=faiss_retriever_module, question='svd')

[[8.4477110e+00 8.4477110e+00 3.4028235e+38]]
[[ 2  3 -1]]


In [64]:
def combine_pos_distance(pos_lists, distance_lists):
    # 合并并去重位置列表
    all_pos = [pos for sublist in pos_lists for pos in sublist if pos > -1]
    combined_pos = sorted(set(all_pos))
    
    # 建立位置-距离映射表
    mappings = []
    for pos_sublist, dist_sublist in zip(pos_lists, distance_lists):
        mappings.append(dict(zip(pos_sublist, dist_sublist)))
    
    # 合并距离列表
    combined_dist = []
    for pos in combined_pos:
        distances = []
        for mapping in mappings:
            if pos in mapping:
                distances.append(mapping[pos])
        avg = round(sum(distances)/len(distances), 3) if distances else 0
        combined_dist.append(avg)
    
    return combined_pos, combined_dist

In [65]:
_conc_pos_list = np.concatenate([_pos_lists_qsrc, _pos_lists_module], axis=0)
print(_conc_pos_list)
_conc_dist_list = np.concatenate([_dist_lists_qsrc, _dist_lists_module], axis=0)
print(_conc_dist_list)

[[ 3 -1 -1]
 [ 2  3 -1]]
[[4.8148732e+00 3.4028235e+38 3.4028235e+38]
 [8.4477110e+00 8.4477110e+00 3.4028235e+38]]


In [66]:
ret_combined_pos, ret_combined_dist = combine_pos_distance(_conc_pos_list, _conc_dist_list)
print(ret_combined_pos)
print(ret_combined_dist)

[2, 3]
[8.448, 6.631]


In [67]:
def getSubMessages(df_row):
    return {
        "description": replace_chinese_punctuation(str(df_row['问题现象描述'])),
        "module": replace_chinese_punctuation(str(df_row['模块'])),
        "severity": replace_chinese_punctuation(str(df_row['严重度'])),
        "cause": replace_chinese_punctuation(str(df_row['原因分析'])),
        "improve": replace_chinese_punctuation(str(df_row['改善对策'])),
        "experience": replace_chinese_punctuation(str(df_row['经验萃取'])),
        "judge": replace_chinese_punctuation(str(df_row['评审后优化'])),
        "score": replace_chinese_punctuation(str(df_row['评分']))
    }

In [87]:
def convert_dr_to_list(row, required_columns):
    # 獲取當前行的所有值
    row_data = []
    max_length = 1  # 預設長度為1
    # 步驟4：檢查是否有列表值並確定最大長度
    for col in required_columns:
        if isinstance(row[col], list):
            max_length = max(max_length, len(row[col]))
    # 步驟5：處理每一列的值
    for i in range(max_length):
        current_row = []
        for col in required_columns:
            value = row[col]
            if isinstance(value, list):
                # 如果是列表，取對應索引的值，如果索引超出範圍則使用空字符串
                current_row.append(value[i] if i < len(value) else '')
            else:
                # 如果不是列表，則重複使用該值
                current_row.append(value)
        row_data.append(current_row)
    return row_data

In [88]:
dfObj_aitrial = pd.read_csv("../../../../doc/DQE_Lesson_learn_AI_0311_five_refine_question_AI_Trial_Run.csv", encoding='utf-8-sig');

In [106]:
def convert_df_to_list(df, pos_list):
    """
    將DataFrame轉換為m*n列表
    Parameters:
    df (pandas.DataFrame): 輸入的DataFrame，包含指定的列
    Returns:
    list: 轉換後的二維列表
    """
    # 步驟1：定義所需的列順序
    required_columns = [
        '模块', '严重度', '问题现象描述', '原因分析', 
        '改善对策', '经验萃取', '评审后优化', '评分'
    ]
    result_list = []
    for idx in pos_list:
        _row = df.iloc[idx]
        result_list.extend(convert_dr_to_list(_row, required_columns));
    return result_list
        # print(f"idx is {idx} and _row type is {type(_row)}:\n")
        # print(_row)
    # 步驟2：初始化結果列表
    # result_list = []
    # 步驟3：遍歷DataFrame的每一行
    # for idx in pos_list:
    #     # 步驟6：將處理後的行數據添加到結果列表
    #     # result_list.extend(row_data)
    #     _row = df.iloc[idx]
    #     if row == None:
    #         print("row variable is None")
    #     result_list.extend(convert_dr_to_list(_row, required_columns));
    # return result_list


In [111]:
datasrclist = convert_df_to_list(dfObj_aitrial, ret_combined_pos)
print(datasrclist)

[['工艺', 'A', '2022年12月12日，Y1 PLUS项目在比亚迪首次量产，按照客户要求首单2K全检，发现C壳的I/O接口位置区域划伤，不良不良10%左右\n\n影响：外观划伤不良需要进行返工，影响量产进度和交付周期', '1.现场确认为USB口边缘划伤，根据此现象确认为USB口插拔过程中刮蹭造成；\n2.根据不良制成排查在DL1、FCT、DL2工位均需要插拔USB，模拟不良使用USB插拔如斜插斜拔会刮蹭到C壳USB口而导致掉漆，正常垂直插拔不易刮蹭到USB口，因此确认为员工作业手法错误导致；', '1.重新培训员工操作技巧和手法，跟进作业效果，要求员工严格按照SOP要求进行自检。\n2.长期对策：壳料来料导入防护美纹胶纸；', '喷漆件塑胶壳I/O接口增加美纹胶保护；\n执行人：结构工程师\n执行节点：模具开模评审\n效果确认：新产品各阶段的试产\n效果输出：试产问题点报告确认是否有I/O接口区域有划伤的问题\n效果确认人：NPI工程师，DQE工程师', '喷漆件塑胶壳I/O接口增加美纹胶保护\n（传音项目100%执行，公版项目逐步推行）', 3], ['工艺', 'A', '2023年7月26日，硬件去比亚迪现场分析主板功能不良，确认发现主板的元器件撞件问题居多，比例约占20%\n\n影响：生产的直通率低，增加维修成本，同时有隐性的售后风险', '设计端做原因分析：目前的设计是元器件距离板边和螺丝柱的距离是1mm或1.5mm，更容易导致撞件风险', '硬件优化设计规范：优化后元器件距离边缘和螺丝柱1.5mm以上；', '元器件距离螺丝柱1.5mm以上；\n执行人：硬件工程师\n执行节点：layout评审\n效果确认：layout手板，EVT，DVT阶段确认', '元器件距离螺丝柱1.5mm以上', 3]]


In [112]:
def generate_markdown_table(headers, value_matrix):
    writer = MarkdownTableWriter(
        tablename="回覆表格",
        headers=headers,
        value_matrix=value_matrix,
        column_alignments=["left"] * len(headers),  # Explicit alignment
        margin=1  # Add margin for better readability
    )
    # Add table style
    writer.style = Style(
        align_header="center",
        border_chars={
            "left": "|",
            "right": "|",
            "header": "-",
            "center": "|",
        }
    )
    return writer.dumps()

In [113]:
headers=["模块", "严重度(A/B/C)", "题现象描述", "原因分析", "改善对策", "经验萃取", "审后优化", "评分"]

In [114]:
ret_md_table = generate_markdown_table(headers=headers, value_matrix=datasrclist);

In [101]:
print(ret_md_table)

| 模块 | 严重度(A/B/C) | 题现象描述 | 原因分析 | 改善对策 | 经验萃取 | 审后优化 | 评分 |
| ---- | ------------- | ---------- | -------- | -------- | -------- | -------- | ---- |

